In [ ]:
import sys
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
sys.path.append('../src')
from modules import (
                    paths,
                    dataset,
                    model,
                    utils,
                    acdc
                    )
from torchvision.transforms import v2

In [ ]:
toy = True
batch_size = 2

In [ ]:
import importlib
importlib.reload(dataset)
if toy == True:
    print("laoding toy datasets")
    train_dataset = dataset.load("train", tiny=True, stop=2)
    val_dataset = dataset.load("valid", tiny=True, stop=2)

else:
    print("loading full dataet")
    train_dataset = dataset.load("train", stop=2)
    val_dataset = dataset.load("valid", stop=2)
print("train:\n"+str(train_dataset))
print("validation:\n"+str(val_dataset))


In [ ]:
transform_train = v2.Compose([
    v2.Lambda(lambda x: x.convert('RGB')),  # some images are in grayscale
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.RandomHorizontalFlip(),
    v2.RandAugment(),
    v2.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    v2.RandomErasing(p=0.25),

])

transform_valid = v2.Compose([
    v2.Lambda(lambda x: x.convert('RGB')),  # some images are in grayscale
    v2.ToImage(), 
    v2.ToDtype(torch.float32, scale=True),
    v2.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225)),
])

In [ ]:
train_loader = DataLoader(
    dataset.TorchDatasetWrapper(train_dataset, transform_train),
    batch_size=batch_size,
    shuffle=True,
    num_workers=2,  
    pin_memory=True,
    prefetch_factor=4,
    persistent_workers=True
)

val_loader = DataLoader(
    dataset.TorchDatasetWrapper(val_dataset, transform_valid),
    batch_size=3,
    shuffle=False,
    num_workers=2,  
    pin_memory=True,
    prefetch_factor=4,
    persistent_workers=True
)

In [ ]:
batch = next(iter(train_loader))

In [ ]:
importlib.reload(model)
config = {
    "patch_size": 8,
    "hidden_size": 48,
    "num_hidden_layers": 4,
    "num_attention_heads": 4,
    "intermediate_size": 4 * 48,
    "hidden_dropout_prob": 0.0,
    "attention_probs_dropout_prob": 0.0,
    "initializer_range": 0.02,
    "image_size": 64,
    "num_classes": 10,
    "num_channels": 3,
    "qkv_bias": True,
}
# embedding = model.Embeddings(config)
# x = embedding(batch[0])
# single = model.AttentionHead(1000, 10, 0.1)
# single(x)
# multi = model.MultiHeadAttention(config)
# multi(x, output_attentions = True)
# encoder = model.Encoder(config)
# encoder(x)
# input = torch.ones(config["hidden_size"])
# bad_input = torch.ones(config["hidden_size"]) +100000
vit = model.ViT(config)

In [ ]:
importlib.reload(utils)

computation_graph = utils.ComputationalGraph(vit, config["num_hidden_layers"], config["num_attention_heads"])
computation_graph.visualize()
print(computation_graph.nodes.keys())

In [ ]:
importlib.reload(acdc)

with acdc.SaveActivations(list(computation_graph.nodes.values())) as ctx:
    vit(batch[0])
    activations = ctx.get_activations()
activations

In [ ]:
importlib.reload(dataset)
animal_dataset, coarse_labels = dataset.load_animal_dataset("train")
animal_dataset = dataset.TorchDatasetWrapper(animal_dataset)

In [ ]:
importlib.reload(dataset)
matching_dataset = dataset.ContrastiveWrapper(animal_dataset, coarse_labels)

In [ ]:
matching_dataset[1]

In [ ]:
importlib.reload(acdc)
with acdc.ReplaceActivations(m.l2, activations["dense2"]):
    with acdc.SaveActivations([(m.l1, "dense1"), (m.l2, "dense2")]) as ctx:
        print(m(torch.Tensor([1])))
        activations1 = ctx.get_activations()
    
print(activations1)